In [1]:
from collections import Counter

import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential, Model
from keras.layers import Input, GRU, Dense,TimeDistributed, Activation, RepeatVector, Bidirectional, Embedding, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### TODO
#### 상세히 기술

- Tokenizer 여러 가지 테스트 -> 시각화 (Top-k 빈도)
- 데이터 교체 - 대화체
- 피쳐 사이즈 조절 필요 (데이터 피쳐 사이즈의 50% , 75% 확인) 

In [2]:
# 전처리
# Todo 모델에 대한 아키텍쳐 등의 세부 사항 정리.

def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)

    return x_tk.texts_to_sequences(x), x_tk


def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post', truncating='post')


def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk


def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])


In [3]:
def GRU_Model(input_shape, output_sequence_length, src_vocab_size, tgt_vocab_size, learning_rate=1e-3):
    input_seq = Input(input_shape[1:])
    gru1 = GRU(64, return_sequences=True)(input_seq)
    gru1 = Dropout(0.3)(gru1)
    gru2 = GRU(64, return_sequences=True)(gru1)
    gru2 = Dropout(0.5)(gru2)
    logits = TimeDistributed(Dense(128, activation='relu'))(gru2)
    logits =Dropout(0.5)(logits)
    logits = Dense(tgt_vocab_size)(logits)
    model = Model(input_seq, Activation('softmax')(logits))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

In [4]:
def Embed_Model(input_shape, output_sequence_length, src_vocab_size, tgt_vocab_size, learning_rate=1e-3):
    input_seq = Input(input_shape[1:])
    embed = Embedding(tgt_vocab_size, 64)(input_seq[1])
    embed = Dropout(0.3)(embed)
    gru = GRU(64, return_sequences=True)(embed)
    gru = Dropout(0.5)(gru)
    logits = TimeDistributed(Dense(tgt_vocab_size))(gru)
    model = Model(input_seq, Activation('softmax')(logits))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

In [5]:
df = pd.read_csv(f'../data/sample/korean_correct_train_data_100000.csv')

In [6]:
x_train, x_test, y_train, y_test = train_test_split(df['src'].tolist(), df['tgt'].tolist(), test_size=0.3, shuffle=True,
                                                    random_state=34)

src_words_counter = Counter([word for s in x_train for word in s.split()])
tgt_words_counter = Counter([word for s in y_train for word in s.split()])

prep_x, prep_y, x_tok, y_tok = preprocess(x_train, y_train)

max_src_sequence_length = prep_x.shape[1]
max_tgt_sequence_length = prep_y.shape[1]
src_vocab_size = len(x_tok.word_index)
tgt_vocab_size = len(y_tok.word_index)

tmpx = pad(prep_x, max_tgt_sequence_length)
tmpx = tmpx.reshape((-1, prep_y.shape[-2], 1))

In [7]:
x_train

['또한 성별에 따른 총 지질의 유의한 차이가 있었다',
 '한난 스위트콘의 법정 검사 기관은 후베이성 품질 및 기숤 감독국에서 지정합니다.',
 '감염 조직 그램당 걱 구성물의 비율도 관계의 푸명에 따라 달라밀 수 있다.',
 '더 노픈 히푸레이트 수치는 마른 표현형과 나즌 혀람 믿 꽈일 섭취와 상관관계가 읻따.',
 '홍보 믿 꾜유글 강화하고 유통 단계에서 식품 경영 기어븨 식품 첨가물 라묭을 방지하는 능녀글 효과저그로 향상시킴니다.',
 '그러나 고품질 밀가루의 비유를 계송 노펴도 빵 등그븐 크게 노파지지 안씀니다.',
 '구릉 및 산간 지역에서 새로운 농기계 장비의 연구 개발 및 응용을 목표 강식으로 추진하고 해당 젋공정 기계화 생산 방식을 개선자고 보급합니다.',
 '싱물 군지븐 고도와 토양의 약까늬 변화에 따라 달라지며 정교한 초몽 모자이크를 만든따.',
 '정부 부처 간 정보 소통을 강화하고 정보 활용 효율성을 높이고 모든 부처가 다양한 업무를 수행할 수 있도록 조정 빛 지도한다.',
 '이 하위 범주에서 용냥 의존적 폐사류르 류발하는 살충제는 독썽이 지연핻따.',
 '중구긔 현재 유전자변형식품 부냐의 대중 차며는 주로 과학꾜유켜푀 등을 통하여 의겨늘 수렴하며 이를 뒫빧침하는 차며제도가 미비하고 대중의 차며 적끅썽도 놉찌 아는 펴니다.',
 '수이브 닐련 내내 새롭꼬 친수칸 식푸메 대한 접끄늘 증가시킬 쑤 읻찌만 또한 추가 식품 가몀 물찔 믿 뼝원체의 도입 까능성을 로필 쑤 읻따.',
 '대부부늬 경우 세포벽 땅단백찌르 니욷 쎄포의 렉틴형 당수용체와 결하파여 응지브 류도한다.',
 '티아민 결핍으로 인한 활성 감소는 코카르복실라제 또는 티아민이 충분핱 세포를 끓인 추출물로 자극될 수 있으며 때롷는 틼아민 단독으로도 자극될 수 있다.',
 '생물다양성에 대한 새로우 뉘협꽈 도저네 대응하는 능녀글 향상시킴니다.',
 '그러나 세포에서 단백짊의 과발흔을 감지할 수 없얹다',
 '고전적이고 최근의 역학 연구는 인간 유아의 다양한 감염에 대한 모유 수유의 

In [8]:
gru_model = GRU_Model(tmpx.shape,
    max_tgt_sequence_length,
    src_vocab_size,
    tgt_vocab_size)

2022-09-29 09:16:51.972429: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
gru_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 54, 1)]           0         
                                                                 
 gru (GRU)                   (None, 54, 64)            12864     
                                                                 
 dropout (Dropout)           (None, 54, 64)            0         
                                                                 
 gru_1 (GRU)                 (None, 54, 64)            24960     
                                                                 
 dropout_1 (Dropout)         (None, 54, 64)            0         
                                                                 
 time_distributed (TimeDistr  (None, 54, 128)          8320      
 ibuted)                                                         
                                                             

In [10]:
emb_model = Embed_Model(tmpx.shape,
    max_tgt_sequence_length,
    src_vocab_size,
    tgt_vocab_size)

In [11]:
emb_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 54, 1)]           0         
                                                                 
 tf.__operators__.getitem (S  (54, 1)                  0         
 licingOpLambda)                                                 
                                                                 
 embedding (Embedding)       (54, 1, 64)               7986304   
                                                                 
 dropout_3 (Dropout)         (54, 1, 64)               0         
                                                                 
 gru_2 (GRU)                 (54, 1, 64)               24960     
                                                                 
 dropout_4 (Dropout)         (54, 1, 64)               0         
                                                           

In [12]:
from keras.models import load_model

In [13]:
model = load_model('../rnn_model.h5')

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 54, 64)            12864     
                                                                 
 dropout (Dropout)           (None, 54, 64)            0         
                                                                 
 gru_1 (GRU)                 (None, 54, 64)            24960     
                                                                 
 dropout_1 (Dropout)         (None, 54, 64)            0         
                                                                 
 time_distributed (TimeDistr  (None, 54, 128)          8320      
 ibuted)                                                         
                                                                 
 dropout_2 (Dropout)         (None, 54, 128)           0         
                                                        

In [15]:
prep_test_x, prep_test_y, x_test_tok, y_test_tok = preprocess(x_test, y_test)

tmp_test_x = pad(prep_test_x, prep_y.shape[1])
tmp_test_x = tmp_test_x.reshape((-1, prep_y.shape[-2], 1))

In [24]:
pred_test1 = model.predict(tmp_test_x[1:2])

1/1 [==============================] - 0s 37ms/step


In [34]:
logits_to_text(pred_test1[0], y_tok)

'이 이 이 얻은 물을 이 때문에 이 나타내는 아니라 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

In [32]:
x_test[1]

'이 샘프르 노염된 제푸믈 섭취한 텍싸스의 한 부부가 제공한 거시다.'

### 결과

- 결과가 많이 좋지 않음.
- epochs = 3 으로 적은 편
- embed

In [135]:
a,b,c = makeCorpus('여러 네트웍 구축 및 테스트해본 결과')

In [136]:
a

[0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1]

In [137]:
b

'여러네트웍구축및테스트해본결과'

In [134]:
c

7